<img src='img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Optimization and Profiling

To appreciate what Anaconda Accelerate provides, it is useful to start by considering some general aspects of optimization and profiling, and to quickly review the profiling modules available in core Python.

## Table of Contents
* [Optimization and Profiling](#Optimization-and-Profiling)
* [Only after the code has been identified...](#Only-after-the-code-has-been-identified...)
* [Profiling Tools in Python](#Profiling-Tools-in-Python)
	* [The Sample Code](#The-Sample-Code)
	* [timeit](#timeit)
	* [cProfile](#cProfile)
	* [`line_profiler`](#line_profiler)
	* [`memory_profiler`](#memory_profiler)
* [Example: Profiling Computation of $\pi$](#Example:-Profiling-Computation-of-$\pi$)
	* [Pure Python Implementation](#Pure-Python-Implementation)
	* [Numpy Implementation](#Numpy-Implementation)
* [Profiling with Accelerate](#Profiling-with-Accelerate)


# Only after the code has been identified...

Profiling answers the questions:

* When do I need to optimize?
* Where do I need to optimize?

A strong arguement can be made that the answer to the first is "only when you have an answer to the second"... and this answer comes from profiling.

> *"We should forget about small efficiencies, say about 97% of the time: Premature optimization is the root of all evil. Yet we should not pass up our opportunities in that critical 3%. A good programmer will not be lulled into complacency by such reasoning, he will be wise to look carefully at the critical code; but only after that code has been identified""
    -- Donald Knuth (who attributed the observation to Tony Hoare)*

# Profiling Tools in Python

Profiling reveals resources that were used to perform the computation: 

* Where was time spent, how much time was spent?
* How much time is spent in each function?
* How many times each function was called?
* How much memory was used?

Different profiling tools help us address different kinds of optimization:

* Time Profiling
  * Functions
  * Lines
* Memory Profiling
* Disk Profiling

Some of the most commonly used profiling tools available in core Python:
 * `%%timeit`
 * `cProfile`
 * `line_profiler`
 * `memory_profiler`

## The Sample Code

We need some sample code to demonstrate various profiler tools.

Outline of sample code:

* create a list based on an input `size`
* iterate from start to end of the list, computing the sum to the end
* sum the sums

In [ ]:
size = 10
ints = list(range(size))
print( ints )

In [ ]:
sums = []

for index in range(size):
    sum_to_end = sum(ints[index:])
    print( sum_to_end )
    sums.append( sum_to_end )

In [ ]:
print(sum(sums))

The sample python file `src/profiling.py` contains such code: load it to inspect the source before applying different profiling tools.

Open <a href='../../edit/Accelerate/src/profiling.py'>./src/profiling.py</a>

Notice the use of `builtins` and the `@profile` decorator

```python
import builtins
profile = getattr(builtins, "profile", lambda x: x)
help(getattr)
```

Notice the use of `__name__`

```python
if __name__ == "__main__":
    sum_of_sums()
```

## timeit

A feature of the Jupyter notebook, `timeit` reports the execution time:

* **`%timeit`** for a single line. (single `%`)
* **`%%timeit`** for an entire code cell. (double `%%`)

In [ ]:
%%timeit

import builtins
profile = getattr(builtins, "profile", lambda x: x)

@profile
def sum_of_sums(size=20000):
    ints = list(range(size))
    sums = []
    for index in range(size):
        sum_to_end=sum(ints[index:])
        sums.append(sum_to_end)

if __name__ == "__main__":
    sum_of_sums()

## cProfile

**`cProfile`** tracks every time a function is called or returns. This means that if you have many calls to short-running functions, `cProfile` will drastically slow down the application.

**`cProfile`** is included in Python and can be used in a number of ways
1. From the command-line to profile and entire application
2. From IPython to profile an application or cell
3. To profile specific snippets of code

Now let's run it again, but this time, we will profile it using `cProfile`:

In [ ]:
!python -m cProfile src/profiling.py

In [ ]:
# An alternate method of running the code from the notebook:
%run -p src/profiling.py

Based on the `tottime` column, it appears to spend the most time in `sum()`, and the second most in the function which wraps to the call to `sum()`.

Instead of relying on he command line, we can profile by adding the import of `cProfile` to the script and construct a `Profile` object.

In [ ]:
import cProfile

def sum_of_sums(size=20000):
    ints = list(range(size))
    sums = []
    for index in range(size):
        sum_to_end=sum(ints[index:])
        sums.append(sum_to_end)

if __name__ == "__main__":
    profiler = cProfile.Profile()
    profiler.enable()

    sum_of_sums()

    profiler.disable()
    profiler.print_stats(sort="tottime")

## `line_profiler`

**`line_profiler`** tracks and reports on each line of code executed, not just function calls.

To install `line_profiler`, uncomment and run the following command:

In [ ]:
# !conda env list

In [ ]:
# !conda install line_profiler -y

The install of `line_profiler` comes with a wrapper script called `kernprof.py`; you may run it as follows:

In [ ]:
!kernprof --help

The simplest way to run `line_profiler` is to use the `kernprof` wrapper as follows:
* create a `file.py` you wish to profile
    * no need for any `@profile` decorator
* run the command **`kernprof -l -v file.py`** in your system shell
    * **`-l`** option will inject the `@profile` decorator into your script builtins
    * **`-v`** option will display timing information

In [ ]:
!kernprof -l -v src/profiling.py

This method will write an intermediate file `profile.py.lprof` to disk. Remove it when you're finsihed profiling:

In [ ]:
import os
import glob
for file in glob.glob("*.lprof"):
    os.remove(file)

An alternate method is to run `line_profiler` as a 2-step process:

1. run **`kernprof`** to generate an `lprof` file
2. run **`line_profiler`** using an `lprof` file as input

In [ ]:
if os.path.exists("./tmp"):
    for file in glob.glob("*.lprof"):
        os.remove(file)
else:
    os.mkdir("./tmp")

In [ ]:
# Step 1:
!kernprof -l -o tmp/profiling.py.lprof src/profiling.py

In [ ]:
# Step 2:
!python -m line_profiler tmp/profiling.py.lprof

Alternatively, the `line_profiler` can be called from the Jupyter Notebook using the `%lprun` magic.

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f sum_of_sums sum_of_sums()

## `memory_profiler`

This pure python module provides:

* reporting on **process** memory consumption
* reporting on **line-by-line** memory consumption

To install `memory_profiler`, uncomment and run the following cells:

In [ ]:
# !conda env list

In [ ]:
# !conda install memory_profiler -y

The line-by-line memory usage mode is used like `line_profiler`:
* first decorate the function you would like to profile with @profile
* then run the script with th python interpreter.

In this case, we use the `-m` flag to tell python to `import` the `memory_profiler` module.

In [ ]:
!python -m memory_profiler src/profiling.py

Loading the `memory_profiler` module enables the `%memit` Jupyter magic

In [ ]:
%load_ext memory_profiler

In [ ]:
%memit sum_of_sums()

How to read the columns in the output:
* **Line #** is the line number of the code that has been profiled
* **Mem usage** is the memory usage of the Python interpreter after that line has been executed. 
* **Increment** is the difference in memory of the current line with respect to the last one. 
* **Line Contents** is the code that has been profiled.

# Example: Profiling Computation of $\pi$

John Wallis in 1655 determined that $\pi$ could be computed as a product of ratios

$$\pi = 2\prod_{i=1}^{\infty}\frac{4i^2}{4i^2-1}$$

Let's profile two different implementations: pure python and numpy, both computing 1 million terms in the product.

## Pure Python Implementation

Inspect the source file before execution:

Open <a href='../../edit/Accelerate/src/piLoop.py'>./src/piLoop.py</a>

In [ ]:
def piLoop(n):
    pi = 2.0
    for i in range(1,n):
        tmp = 4*i**2
        pi*=tmp/(tmp-1)
    return pi

N=int(1e6)
piLoop(N)

Profiling the total execution time with `timeit`:

In [ ]:
%timeit piLoop(int(1e6))

Profiling function calls with `cProfile`:

In [ ]:
!python -m cProfile src/piLoop.py 1000000

Profiling every line with `line_profiler`:

In [ ]:
!kernprof -l -o tmp/piLoop.py.lprof src/piLoop.py 1000000

In [ ]:
!python -m line_profiler tmp/piLoop.py.lprof

In [ ]:
%lprun -f piLoop piLoop(int(1e6))

Profiling memory usage with `memory_profiler`:

In [ ]:
!python -m memory_profiler src/piLoop.py 1000000

## Numpy Implementation

The point of vectorizing with NumPy is to be able to...
1. apply the same operation to a bunch of input...
2. efficiently (i.e. without having to manually and inefficiently loop over the input) ...
3. and quickly (many NumPy operations are implemented in C)

Other reasons to use NumPy and vectorization
1. More efficient use of memory
  * Python lists are pointers "all the way down"
2. Direct access to the data
  * Python lists are actually lists of pointers to thing
  * NumPy arrays actually contain the data
3. Simplified code

In [ ]:
import numpy as np

def piArray(n):
    series=np.arange(1,n)**2*4.
    series/=(series-1)
    return 2.*series.prod()

piArray(int(1e6))

Profiling the total execution time with `timeit`:

In [ ]:
%timeit piArray(int(1e6))

In order to profile with `cProfile` and others, we need to add some code and write to disk:

In [ ]:
%%writefile tmp/numpy_piArray.py

import sys
import builtins
profile = getattr(builtins, "profile", lambda x: x)

import numpy as np

@profile
def piArray(n):
    series=np.arange(1,n)**2*4.
    series/=(series-1)
    return 2.*series.prod()

if __name__ == "__main__":
    print(piArray(int(sys.argv[1])))


Profiling the funciton call times of the numpy implementation with `cProfile`:

In [ ]:
!python -m cProfile tmp/numpy_piArray.py 1000000

Profiling the time spent per line of code in the numpy implementation with `line_profiler`:

In [ ]:
!kernprof -l -o tmp/numpy_piArray.py.lprof tmp/numpy_piArray.py 1000000

In [ ]:
!python -m line_profiler tmp/numpy_piArray.py.lprof

Profiling the memory usage of the numpy implementation with `memory_profiler`:

In [ ]:
!python -m memory_profiler tmp/numpy_piArray.py 100000

For a comparison of the memory use differences, let's look at a python list versus a numpy ndarray:

In [ ]:
# Use of memory:

import sys
import numpy as np

size = 1000000
l = list(range(1000000))
print("Bytes used by list:", sys.getsizeof(l))
print("Plus bytes used by list elements:", sys.getsizeof(l[0])*len(l))
ltot = sys.getsizeof(l) + (sys.getsizeof(l[0])*len(l))
print("Total:", ltot)

arr = np.array(l, dtype='i4')
print("\nBytes used by array elements:", arr.nbytes)
print("Plus bytes used by of array:", sys.getsizeof(arr))
arrtot = arr.nbytes + sys.getsizeof(arr)
print("Total:", arrtot)

print("")
print("The list is", ltot-arrtot,"bytes bigger")
print("Or, the list is", (ltot-arrtot)/arrtot, "times bigger than the array")

# Profiling with Accelerate

The `accelerate` package contains a `profiler` module which extends the profiling functionality found in core python. 

We will see this demonstrated in the next notebook.

---

<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*